# ECSN Test

In [1]:
from StarKiller.initialization import starkiller_initialize
from StarKiller.interfaces import EosType
from StarKiller.interfaces import BurnType
from StarKiller.eos import Eos
from StarKiller.network import Network
import numpy as np

In [2]:
probin_file = "probin_ecsn"

In [3]:
starkiller_initialize(probin_file)


Initialized StarKiller with ...
- EOS:     helmholtz
- Network: pynucastro


In [4]:
helmholtz = Eos()
ecsn = Network()

In [5]:
def get_eps_nuc_eos(burn_state):
    ecsn.rhs(burn_state)
    print(burn_state.state)
    
    eos_state = burn_state.to_eos_type()
    helmholtz.evaluate(eos_state.eos_input_rt, eos_state)
    print(eos_state.state)
    
    Hnuc = burn_state.ydot[ecsn.net_ienuc]
    print("Energy generation rate: {:0.09e}".format(Hnuc))
    
    aion = ecsn.aion
    zion = ecsn.zion

    # dX/dt = dY/dt * Aion
    omegadot = burn_state.ydot[:ecsn.nspec] * aion
    print("omegadot = {}".format(omegadot))
    
    cp = eos_state.state.cp
    print("cp = {:0.05e}".format(cp))
    
    dpdt = eos_state.state.dpdt
    print("dpdt = {:0.05e}".format(dpdt))
    
    dpdx = eos_state.xderivs.dpdx
    print("dpdx = {}".format(dpdx))
    
    dhdx = eos_state.xderivs.dhdx
    print("dhdx = {}".format(dhdx))

    print("cp * dpdx/dpdt = {}".format(cp * dpdx/dpdt))
    print("cp * dpdx/dpdt * wdot = {:0.09e}".format(np.dot(cp * dpdx/dpdt, omegadot)))
    print("dhdx = {}".format(dhdx))
    print("dhdx * wdot = {:0.09e}".format(np.dot(dhdx, omegadot)))

    dHcompdx = (cp * dpdx/dpdt - dhdx)
    print("dHcompdx = {}".format(dHcompdx))
    
    Hcomp = np.dot(dHcompdx, omegadot)
    print("Composition energy generation: {:0.09e}".format(Hcomp))

    dedx = eos_state.xderivs.dedx
    print("dedx = {}".format(dedx))
    
    de = -np.dot(dedx, omegadot)
    print("eps_nuc_eos = dedx * wdot (cf. MESA) = {:0.09e}".format(de))

    dabardX = (eos_state.state.abar/aion) * (aion - eos_state.state.abar)
    dzbardX = (eos_state.state.abar/aion) * (zion - eos_state.state.zbar)

    dabardt = np.dot(dabardX, omegadot)
    dzbardt = np.dot(dzbardX, omegadot)

    print("dabar/dt = {:0.09e}".format(dabardt))
    print("dzbar/dt = {:0.09e}".format(dzbardt))

    print("sum omegadot = {:0.16e}".format(np.sum(omegadot)))
    print("sum X = {:0.16e}".format(np.sum(burn_state.state.xn)))
    
    Htot = Hnuc + Hcomp
    print("Hnuc + Hcomp (Maestro): {:0.9e}".format(Htot))

## Test 1 from Josiah

In [6]:
burn_state_1 = BurnType()

burn_state_1.state.rho = 7607783959.50421e0
burn_state_1.state.t = 983559309.749412e0
burn_state_1.state.xn[ecsn.species_map["h1"]] = 0.0
burn_state_1.state.xn[ecsn.species_map["he4"]] = 1.14998095787427E-18
burn_state_1.state.xn[ecsn.species_map["o16"]] = 0.5955320774
burn_state_1.state.xn[ecsn.species_map["o20"]] = 0.203902681
burn_state_1.state.xn[ecsn.species_map["f20"]] = 1.17278643621995E-07
burn_state_1.state.xn[ecsn.species_map["ne20"]] = 0.19753136
burn_state_1.state.xn[ecsn.species_map["mg24"]] = 0.0
burn_state_1.state.xn[ecsn.species_map["al27"]] = 0.0
burn_state_1.state.xn[ecsn.species_map["si28"]] = 0.0027259815
burn_state_1.state.xn[ecsn.species_map["p31"]] = 0.0
burn_state_1.state.xn[ecsn.species_map["s32"]] = 0.0

In [7]:
get_eps_nuc_eos(burn_state_1)

<burn_t>{
    rho : 7607783959.50421,
    t : 983559309.749412,
    e : 2.4907431337903124e+18,
    xn : array([0.00000000e+00, 1.14998096e-18, 5.95532077e-01, 2.03902681e-01,
       1.17278644e-07, 1.97531360e-01, 0.00000000e+00, 0.00000000e+00,
       2.72598150e-03, 0.00000000e+00, 0.00000000e+00]),
    cv : 14444247.5529241,
    cp : 14486626.878187362,
    y_e : 0.47945583462538965,
    eta : 87.75746182063078,
    cs : 1099451413.7881768,
    dx : 0.0,
    abar : 17.42469332287015,
    zbar : 8.354370880208164,
    t_old : 0.0,
    dcvdt : 0.0,
    dcpdt : 0.0,
    self_heat : 0,
    i : 0,
    j : 0,
    k : 0,
    n_rhs : 0,
    n_jac : 0,
    time : 0.0,
    success : 0}
<eos_t>{
    rho : 7607783959.50421,
    t : 983559309.749412,
    p : 6.876490585951449e+27,
    e : 2.4907431337903124e+18,
    h : 3.394618772551167e+18,
    s : 46493486.57937588,
    xn : array([0.00000000e+00, 1.14998096e-18, 5.95532077e-01, 2.03902681e-01,
       1.17278644e-07, 1.97531360e-01, 0.000000

## Test 2 from Josiah

In [8]:
burn_state_2 = BurnType()

burn_state_2.state.rho = 8346403698.95185e0
burn_state_2.state.t = 656536305.140964e0
burn_state_2.state.xn[ecsn.species_map["h1"]] = 0.0
burn_state_2.state.xn[ecsn.species_map["he4"]] = 8.89957317703129E-23
burn_state_2.state.xn[ecsn.species_map["o16"]] = 0.5999942706e0
burn_state_2.state.xn[ecsn.species_map["o20"]] = 0.3926652894e0
burn_state_2.state.xn[ecsn.species_map["f20"]] = 0.000000002e0
burn_state_2.state.xn[ecsn.species_map["ne20"]] = 0.0073364571e0
burn_state_2.state.xn[ecsn.species_map["mg24"]] = 0.0
burn_state_2.state.xn[ecsn.species_map["al27"]] = 0.0
burn_state_2.state.xn[ecsn.species_map["si28"]] = 3.518236231242E-06
burn_state_2.state.xn[ecsn.species_map["p31"]] = 0.0
burn_state_2.state.xn[ecsn.species_map["s32"]] = 0.0

In [9]:
get_eps_nuc_eos(burn_state_2)

<burn_t>{
    rho : 8346403698.95185,
    t : 656536305.140964,
    e : 2.436650301745047e+18,
    xn : array([0.00000000e+00, 8.89957318e-23, 5.99994271e-01, 3.92665289e-01,
       2.00000000e-09, 7.33645710e-03, 0.00000000e+00, 0.00000000e+00,
       3.51823623e-06, 0.00000000e+00, 0.00000000e+00]),
    cv : 13444158.983332044,
    cp : 13470518.44039057,
    y_e : 0.46073323962811563,
    eta : 133.98527882572265,
    cs : 1086123519.6557846,
    dx : 6.9010554739524e-310,
    abar : 17.391348207658748,
    zbar : 8.012772201215236,
    t_old : 6.9010554732781e-310,
    dcvdt : 6.9010554732813e-310,
    dcpdt : 6.9010821697651e-310,
    self_heat : -1465952976,
    i : 32521,
    j : -2008587760,
    k : 32521,
    n_rhs : -1465621584,
    n_jac : 32521,
    time : 6.90107576097953e-310,
    success : -1465951120}
<eos_t>{
    rho : 8346403698.95185,
    t : 656536305.140964,
    p : 7.364917477765705e+27,
    e : 2.436650301745047e+18,
    h : 3.3190563946459453e+18,
    s : 404744